In [3]:
# neural networks are constructed with torch.nn package
# torch.autograd train torch.nn
# nn.module: contains layers
# forward(input): forward propagation 
# backprob with direct gradient:weight = weight - learning_rate * gradient

# define neural network 
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
help(Variable.view)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)
Help on function view in module torch.autograd.variable:

view(self, *sizes)



In [2]:
# forward function is defined by myself and backward function is 
# automatically defined by autograd
# the learnable parameters are returned by net.parameters()

params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1 weight [out_channel,in_channel,size]
print(params[2].size()) # conv2 weight
# print(params[0])

10
torch.Size([6, 1, 5, 5])
torch.Size([16, 6, 5, 5])


In [3]:
# the input and output of NN are both autograd.Variable
input = Variable(torch.randn(1,1,32,32))
print(input)

Variable containing:
(0 ,0 ,.,.) = 
 -0.2462 -1.3573  2.2946  ...  -0.0505 -0.3896  1.0694
 -0.0446 -0.9110  0.8707  ...  -1.7768  0.8228 -0.7090
  0.2565  1.2190  0.3005  ...   0.7434  0.0394  0.3056
           ...             ⋱             ...          
 -1.5350  0.2739 -0.9721  ...  -0.0098  0.9692 -0.5719
 -0.6183  1.5776  0.2210  ...   0.2103  1.1442  0.4469
  1.8332  1.3993  1.1494  ...  -0.4118  0.9996  0.5227
[torch.FloatTensor of size 1x1x32x32]



In [4]:
out = net(input)
print(out)

Variable containing:
-0.0700 -0.0449 -0.0264 -0.0286 -0.0369 -0.0023 -0.0357 -0.1086  0.0340 -0.0314
[torch.FloatTensor of size 1x10]



In [5]:
# zero the gradient buffers of all parameters and backprops with random gradients 
net.zero_grad()
out.backward(torch.randn(1,10)) #backprop from random(1,10)

In [6]:
# torch.nn only supports mini-batch of samples, not single samples
# e.g. nn.Conv2d takes 4D tensor as nSamples x nChannels x Height x Width
# for single sample, use input.unsqueeze(0) to add fake batch dimension

# recap of classes
# torch.tensor: multi-dim array
# autograd.Variable: warps a tensor and record history of operations applied to it
# nn.Module: neural network module 
# nn.Parameter: automatically register parameters of neural network 
# autograd.Function: implement forward and backward definitions of a autograd operation

In [9]:
# loss function(here is MSE loss) 
output = net(input)
target = Variable(torch.arange(1,11)) # 1:10
print(target) # assume this is ground-truth 
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

Variable containing:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
[torch.FloatTensor of size 10]

Variable containing:
 38.8384
[torch.FloatTensor of size 1]



In [12]:
# steps backward
print(loss.grad_fn) # MSE Loss
print(loss.grad_fn.next_functions[0][0]) #FC layer 
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [13]:
# use loss.backward() to backpropagate the error 
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  4.7322
  4.1020
  3.5328
 -6.9231
 -3.9706
  2.4370
[torch.FloatTensor of size 6]



In [14]:
# use SGD(default method) to update weights: 
# weight = weight - learning_rate * gradient 
learning_rate = 0.1
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate) # subtraction of weights 

In [15]:
# import module torch.optim to select optimization method 
import torch.optim as optim 

# create optimizer
optimizer = optim.SGD(net.parameters(),lr = 0.01)

# in trainning loop 
optimizer.zero_grad() # zero gradient buffer 
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step() #update weights 

In [5]:
string = [1,2,3,4,5,6]
print(string[0:1:-1])

[]
